In [1]:
import json
import pandas as pd
import importlib
import pickle

In [2]:
filepath = '../scraping/articles/filtered_articles.json'
with open(filepath, 'r') as file:
    articles = pd.read_json(filepath)

filepath = '../relation_extraction/directory.pkl'

with open(filepath, 'rb') as file:
    keywords = pickle.load(file)

In [78]:
import KnowledgeExtractor
importlib.reload(KnowledgeExtractor)

extractor = KnowledgeExtractor.KnowledgeExtractor('dslim/bert-base-NER', keywords)

                ___              __         
               /\_ \      __    /\ \        
 _ __     __   \//\ \    /\_\   \ \ \/'\    
/\`'__\ /'__`\   \ \ \   \/\ \   \ \ , <    
\ \ \/ /\  __/    \_\ \_  \ \ \   \ \ \\`\  
 \ \_\ \ \____\   /\____\  \ \_\   \ \_\ \_\
  \/_/  \/____/   \/____/   \/_/    \/_/\/_/
                                            
                                            



/Users/tiril/Documents/IndividualProject/nuclear_repo/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2024-08-06 11:51:07,074] [INFO] [relik.inference.annotator.from_pretrained:694] [PID:29125] [RANK:0] Loading Relik from sapienzanlp/relik-relation-extraction-nyt-large
[2024-08-06 11:51:07,075] [INFO] [relik.inference.annotator.from_pretrained:695] [PID:29125] [RANK:0] {
    '_target_': 'relik.inference.annotator.Relik',
    'index': {
        'triplet': {
            '_target_': 'relik.retriever.indexers.inmemory.InMemoryDocumentIndex.from_pretrained',
            'name_or_path': 'sapienzanlp/relik-retriever-small-nyt-document-index',
        },
    },
    'metadata_fields': [],
    'reader': {
        '_target_': 'relik.reader.pytorch_modules.triplet.RelikReaderForTripletExtraction',
        'transformer_model': 'sapienzanlp/relik-reader-deberta-v3-large-nyt',
    },
    'retriever': {
        'triplet': {
            '_target_': 'relik.retriever.pytorch_modules.model.GoldenRetriever',
            'question_encoder': 'sapienzanlp/relik-retriever-small-nyt-question-encoder',
        

In [87]:
output = []
for i in range(len(articles)):
    instance = articles.iloc[i]
    url = instance['url']
    sents = instance['text']
    entities = extractor.extract_entities(sents)
    triplets = extractor.extract_relik_triplets(sents)
    output.append({
        'url': url,
        'triplets': triplets,
        'entities': entities
    })
    if i >= 2:
        break

print(len(output))

with open('entities_and_triplets.json', 'w') as f:
    json.dump(output, f, indent=4)

[2024-08-06 12:05:52,253] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 20 number of elements processed
[2024-08-06 12:06:13,949] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 16 number of elements processed


In [ ]:
for o in output:
    print(o['url'])
    print(o['triplets'])
    print(o['entities'])

{'url': 'https://www.nucnet.org/news/kyrgyzstan-6-5-2024', 'triplets': [{'head': 'Kul Province', 'relation': 'contains', 'tail': 'Issyk', 'confidence': 1.0}, {'head': 'Asia', 'relation': 'contains', 'tail': 'Soviet Union', 'confidence': 1.0}, {'head': 'EBRD', 'relation': 'company', 'tail': 'European Bank for Reconstruction and Development', 'confidence': 1.0}, {'head': 'Asia', 'relation': 'contains', 'tail': 'Central Asia', 'confidence': 0.9100000262260437}, {'head': 'Asia', 'relation': 'contains', 'tail': 'Kyrgyzstan', 'confidence': 1.0}, {'head': 'Asia', 'relation': 'contains', 'tail': 'Soviet Union', 'confidence': 1.0}], 'entities': [{'word': 'Bishkek', 'entity': 'ORG'}, {'word': 'Kyrgyzstan', 'entity': 'LOC'}, {'word': 'Kyrgyz', 'entity': 'LOC'}, {'word': 'nuclear power plant', 'entity': 'REACTOR'}, {'word': 'Sadyr Japarov', 'entity': 'PER'}, {'word': 'Issyk-Kul Province', 'entity': 'LOC'}, {'word': 'Kyzyl-Ompol', 'entity': 'LOC'}, {'word': 'Japarov', 'entity': 'PER'}, {'word': 'Ky

In [67]:
def extract_relik_triplets(sents):
    extracted_triplets = []

    relik_out: RelikOutput = relik(sents)

    for output in relik_out:
        output_dict = asdict(output)
        triplets = output_dict['triplets']
        if triplets:
            for triplet in triplets:
                head = triplet.subject.text
                tail = triplet.object.text
                label = triplet.label
                confidence = triplet.confidence
                extracted_triplets.append({
                    'head': head,
                    'relation': label,
                    'tail': tail,
                    'confidence': confidence
                }) 
    
    return extracted_triplets

In [68]:
sents = articles.iloc[0]['text']
triplets = extract_relik_triplets(sents)

[2024-08-06 11:28:04,415] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 20 number of elements processed


In [69]:
for t in triplets:
    print(t)

{'head': 'Kul Province', 'relation': 'contains', 'tail': 'Issyk', 'confidence': 1.0}
{'head': 'Asia', 'relation': 'contains', 'tail': 'Soviet Union', 'confidence': 1.0}
{'head': 'EBRD', 'relation': 'company', 'tail': 'European Bank for Reconstruction and Development', 'confidence': 1.0}
{'head': 'Asia', 'relation': 'contains', 'tail': 'Central Asia', 'confidence': 0.9100000262260437}
{'head': 'Asia', 'relation': 'contains', 'tail': 'Kyrgyzstan', 'confidence': 1.0}
{'head': 'Asia', 'relation': 'contains', 'tail': 'Soviet Union', 'confidence': 1.0}
